In [1]:
# !pip install google-play-scraper
# !pip install nltk

     ---------------------------------------- 1.5/1.5 MB 330.7 kB/s eta 0:00:00
     ------------------------------------ 263.0/263.0 kB 283.7 kB/s eta 0:00:00
     -------------------------------------- 96.6/96.6 kB 690.8 kB/s eta 0:00:00
     -------------------------------------- 78.4/78.4 kB 396.6 kB/s eta 0:00:00


In [53]:
from google_play_scraper import app, reviews, Sort
import json

In [54]:
result, continuation_token = reviews('com.pandora.android',
                                    lang='en',
                                    country='us',
                                    sort = Sort.NEWEST,
                                    count = 532)

result, _ = reviews('com.pandora.android', 
                   continuation_token = continuation_token)

In [55]:
import pandas as pd
import numpy as np
import nltk

In [82]:
df = pd.DataFrame(result)
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,a0840420-93a4-482f-ac7f-7df763f80673,Gerry Champoux,https://play-lh.googleusercontent.com/a-/AFdZu...,"Up until the August 1 release, this was my go-...",3,0,2208.1,2022-08-22 00:17:58,None,NaT
1,767bfa86-8af7-4c8b-a4fa-6d5b9aa40f82,Harley,https://play-lh.googleusercontent.com/a-/AFdZu...,Everytime I try to connect it to my Google thr...,1,0,2208.1,2022-08-21 23:48:04,None,NaT
2,5d4df15f-61bf-412d-b39c-0ad22613d3ab,Hardcore Master,https://play-lh.googleusercontent.com/a-/AFdZu...,Terrible,1,0,None,2022-08-21 23:25:05,None,NaT
3,0d540f29-358c-4ebb-8a03-2b8fa650d786,Cheri Boyer,https://play-lh.googleusercontent.com/a-/AFdZu...,"All these years, now I have to listen to celeb...",1,0,2205.1.1,2022-08-21 23:16:34,None,NaT
4,4b19334f-7e19-4ed0-8b89-09e0b525d86f,Tim Kelley,https://play-lh.googleusercontent.com/a-/AFdZu...,Crashes 100% of the time on Pixel 6 pro Androi...,1,0,2205.1.1,2022-08-21 23:06:27,"Hi Tim, Sorry to hear that! Please try running...",2022-08-23 02:25:51


In [83]:
df = df.drop(columns=['userName',
                      'userImage',
                      'thumbsUpCount',
                      'reviewCreatedVersion',
                      'at',
                      'replyContent',
                      'repliedAt'])
df.head()

,reviewId,content,score
0,a0840420-93a4-482f-ac7f-7df763f80673,"Up until the August 1 release, this was my go-...",3
1,767bfa86-8af7-4c8b-a4fa-6d5b9aa40f82,Everytime I try to connect it to my Google thr...,1
2,5d4df15f-61bf-412d-b39c-0ad22613d3ab,Terrible,1
3,0d540f29-358c-4ebb-8a03-2b8fa650d786,"All these years, now I have to listen to celeb...",1
4,4b19334f-7e19-4ed0-8b89-09e0b525d86f,Crashes 100% of the time on Pixel 6 pro Androi...,1


In [87]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
df['score'] = scaler.fit_transform(df['score'].values.reshape(-1, 1))

0      0.5
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
527    1.0
528    1.0
529    1.0
530    1.0
531    1.0
Name: score, Length: 532, dtype: float64

In [91]:
df['score'] = np.where(df['score'] >= 0.5, 1, 0)

,reviewId,content,score
0,a0840420-93a4-482f-ac7f-7df763f80673,"Up until the August 1 release, this was my go-...",1
1,767bfa86-8af7-4c8b-a4fa-6d5b9aa40f82,Everytime I try to connect it to my Google thr...,0
2,5d4df15f-61bf-412d-b39c-0ad22613d3ab,Terrible,0
3,0d540f29-358c-4ebb-8a03-2b8fa650d786,"All these years, now I have to listen to celeb...",0
4,4b19334f-7e19-4ed0-8b89-09e0b525d86f,Crashes 100% of the time on Pixel 6 pro Androi...,0


In [88]:
import nltk
 
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re

[nltk_data] Downloading package stopwords to C:\Users\Sankeerth
[nltk_data]     Erra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [89]:
corpus = []
reviews = df['content']
for i in reviews:
    review = re.sub('[^a-zA-Z]', ' ',i)
    review = review.lower()
    review = review.split()

    ps = PorterStemmer()
 
    review = [ps.stem(word) for word in review
                if not word in set(stopwords.words('english'))]

    review = ' '.join(review) 
    corpus.append(review)
# print(corpus)

In [90]:
df.head()

,reviewId,content,score
0,a0840420-93a4-482f-ac7f-7df763f80673,"Up until the August 1 release, this was my go-...",0.5
1,767bfa86-8af7-4c8b-a4fa-6d5b9aa40f82,Everytime I try to connect it to my Google thr...,0.0
2,5d4df15f-61bf-412d-b39c-0ad22613d3ab,Terrible,0.0
3,0d540f29-358c-4ebb-8a03-2b8fa650d786,"All these years, now I have to listen to celeb...",0.0
4,4b19334f-7e19-4ed0-8b89-09e0b525d86f,Crashes 100% of the time on Pixel 6 pro Androi...,0.0


In [94]:
from sklearn.feature_extraction.text import CountVectorizer
 
dataset = df
    
cv = CountVectorizer(max_features = 1500)

X = cv.fit_transform(corpus).toarray()
 
y = dataset.iloc[:, 2].values

In [96]:
from sklearn.model_selection import train_test_split
 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [97]:
from sklearn.ensemble import RandomForestClassifier
 
model = RandomForestClassifier(n_estimators = 501,
                            criterion = 'entropy')
                             
model.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=501)

In [98]:
y_pred = model.predict(X_test)
 
y_pred

array([1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       1])

In [102]:
from sklearn.metrics import confusion_matrix, accuracy_score
 
cm = confusion_matrix(y_test, y_pred)
ac = accuracy_score(y_test, y_pred)
print("Confusion Matrix: \n", cm,"\n", "Accuracy: ",  ac)

Confusion Matrix: 
 [[19 15]
 [12 87]] 
 Accuracy:  0.7969924812030075
